#### Notebook to feature engineer fraction of time PM10 was above EPA standard for all counties

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2008','2008_single_level_34_variables.nc')
sl_34_variables = xr.open_dataset(PATH_1) ## single level 34 variables
sl_34_variables
pm10=sl_34_variables['pm10'] ## get only pm2.5 variable
pm10

<xarray.DataArray 'pm10' (valid_time: 2928, latitude: 34, longitude: 78)> Size: 31MB
[7765056 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2008-01-01 ... 2008-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/33)
    GRIB_paramId:                             210074
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               kg m**-3
    long_name:                                Particulate matter d <= 10 um
    units:                                    kg m**-3
    standard_name:                            mass_concentration_of_pm10_ambi...
    GRIB_number:                              0
    GRIB_surface:                             0.0

In [3]:
pm10.to_dataframe()

pm10
valid_time          latitude longitude                              
2008-01-01 00:00:00 49.25    -124.849999999999994  0.000000007753970
                             -124.099999999999994  0.000000013731048
                             -123.349999999999994  0.000000018631605
                             -122.599999999999994  0.000000015812043
                             -121.849999999999994  0.000000009474556
...                                                              ...
2008-12-31 21:00:00 24.50    -70.099999999999994   0.000000002054186
                             -69.349999999999994   0.000000001414755
                             -68.599999999999994   0.000000000814701
                             -67.849999999999994   0.000000000054932
                             -67.099999999999994   0.000000000000000

[7765056 rows x 1 columns]

In [4]:
pm_threshold=150*10**(-9) ## epa standard
pm_threshold

1.5000000000000002e-07

In [5]:
## use county shapefile from 2008 as the shapefile before 2008 could not be found
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,geometry
0,19,175,00465276,19175,Union,Union County,06,H1,G4020,None,None,None,A,"POLYGON ((-94.47051 40.97504, -94.47050 40.975..."
1,19,177,00465277,19177,Van Buren,Van Buren County,06,H1,G4020,None,None,None,A,"POLYGON ((-91.96059 40.90070, -91.95925 40.900..."
2,20,097,00485013,20097,Kiowa,Kiowa County,06,H1,G4020,None,None,None,A,"POLYGON ((-99.01471 37.67895, -99.01473 37.678..."
3,20,109,00485019,20109,Logan,Logan County,06,H1,G4020,None,None,None,A,"POLYGON ((-100.89562 39.13329, -100.89404 39.1..."
4,20,003,00484971,20003,Anderson,Anderson County,06,H1,G4020,None,None,None,A,"POLYGON ((-95.51741 38.20619, -95.51741 38.206..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,35,015,00936829,35015,Eddy,Eddy County,06,H1,G4020,None,16100,None,A,"POLYGON ((-104.09554 32.00000, -104.09586 32.0..."
3229,40,015,01101795,40015,Caddo,Caddo County,06,H1,G4020,None,None,None,A,"POLYGON ((-98.36984 35.55133, -98.36964 35.551..."
3230,40,025,01101800,40025,Cimarron,Cimarron County,06,H1,G4020,None,None,None,A,"POLYGON ((-102.14252 36.50032, -102.14380 36.5..."
3231,40,069,01101822,40069,Johnston,Johnston County,06,H1,G4020,None,None,None,A,"POLYGON ((-96.74415 34.17223, -96.74416 34.172..."


In [6]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-94.47051 40.97504, -94.47050 40.975...
1       POLYGON ((-91.96059 40.90070, -91.95925 40.900...
2       POLYGON ((-99.01471 37.67895, -99.01473 37.678...
3       POLYGON ((-100.89562 39.13329, -100.89404 39.1...
4       POLYGON ((-95.51741 38.20619, -95.51741 38.206...
                              ...                        
3228    POLYGON ((-104.09554 32.00000, -104.09586 32.0...
3229    POLYGON ((-98.36984 35.55133, -98.36964 35.551...
3230    POLYGON ((-102.14252 36.50032, -102.14380 36.5...
3231    POLYGON ((-96.74415 34.17223, -96.74416 34.172...
3232    POLYGON ((-84.27514 39.28921, -84.27467 39.289...
Name: geometry, Length: 3233, dtype: geometry

In [7]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0    -94.470506000000000  40.975042999999999
     1    -94.470500999999999  40.975550999999996
     2    -94.470511999999999  40.978164000000000
     3    -94.470497999999992  40.978803999999997
     4    -94.470501999999996  40.979332999999997
...                       ...                 ...
3232 3894 -84.275637000000003  39.289242000000002
     3895 -84.275549999999996  39.289235999999995
     3896 -84.275515999999996  39.289234000000000
     3897 -84.275508000000002  39.289232999999996
     3898 -84.275143000000000  39.289206999999998

[7781581 rows x 2 columns]

#### Fraction of time PM10 was above EPA threshold

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time, PM2.5 was above EPA standard
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= pm10.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['pm10'].values[i] > pm_threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'PM10 above threshold':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 28min 28s
Wall time: 29min 11s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,PM10 above threshold
0,0.000000000000000
1,0.000000000000000
2,0.068306010928962
3,0.000000000000000
4,0.307377049180328
...,...
3228,0.000000000000000
3229,0.000000000000000
3230,0.000000000000000
3231,0.000000000000000


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop([ 'STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD','LSAD','CLASSFP','MTFCC',
                                 'CSAFP','CBSAFP','METDIVFP','FUNCSTAT'], axis=1)
county_gdf

,NAME,geometry,fips
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003
...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,PM10 above threshold
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,0.000000000000000
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,0.000000000000000
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,0.068306010928962
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,0.000000000000000
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,0.307377049180328
...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,0.000000000000000
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,0.000000000000000
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,0.000000000000000
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,0.000000000000000


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,PM10 above threshold
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,0.000000000000000
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,0.000000000000000
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,0.068306010928962
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,0.000000000000000
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,0.307377049180328


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,PM10 above threshold
0,Union,19175,0.000000000000000
1,Van Buren,19177,0.000000000000000
2,Kiowa,20097,0.068306010928962
3,Logan,20109,0.000000000000000
4,Anderson,20003,0.307377049180328
...,...,...,...
3228,Eddy,35015,0.000000000000000
3229,Caddo,40015,0.000000000000000
3230,Cimarron,40025,0.000000000000000
3231,Johnston,40069,0.000000000000000


In [16]:
county_var.to_pickle('PM10_above_threshold.pkl')